In [1]:
import pandas as pd
import numpy as np
import warnings
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import classification_report
from imblearn.pipeline import Pipeline, make_pipeline
from imblearn.over_sampling import RandomOverSampler


trainCSV = "https://raw.githubusercontent.com/andybbruno/DataMining/master/new_train_cleaned.csv?token=AGWKQXZRZZDVJOUEMOQ3PJC6AHO6U"

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: DeprecationWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", DeprecationWarning)


In [2]:
# READ AND DROP COLUMNS
df = pd.read_csv(trainCSV)
y = df['IsBadBuy']
X = df.drop(columns=['IsBadBuy'])

# Split the dataset in two equal parts
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=0)

# Set the parameters by cross-validation
params = {
    'n_estimators': [25, 50, 100, 200, 500, 1000]
}

new_params = {'randomforestclassifier__' + key: params[key] for key in params}

#scores = ['precision', 'recall', 'f1']
scores = ['recall', 'f1']

for score in scores:
    print("# Tuning hyper-parameters for ----> %s" % score)
    print()
    
    obj = make_pipeline(RandomOverSampler(), RandomForestClassifier())
    
    if (score == "recall"):
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring=score)
    else:
      clf = GridSearchCV(obj, param_grid=new_params, cv=5, scoring='%s_macro' % score)
                         
    clf.fit(X_train, y_train)

    print("Best parameters set found on development set:")
    print()
    print(clf.best_params_)
    print()
    print("Grid scores on development set:")
    print()
    means = clf.cv_results_['mean_test_score']
    stds = clf.cv_results_['std_test_score']
    for mean, std, params in zip(means, stds, clf.cv_results_['params']):
        print("%0.3f (+/-%0.03f) for %r"
              % (mean, std * 2, params))
    print()

    print("Detailed classification report:")
    print()
    print("The model is trained on the full development set.")
    print("The scores are computed on the full evaluation set.")
    print()
    y_true, y_pred = y_test, clf.predict(X_test)
    print(classification_report(y_true, y_pred))
    print()

# Tuning hyper-parameters for ----> recall

Best parameters set found on development set:

{'randomforestclassifier__n_estimators': 50}

Grid scores on development set:

0.258 (+/-0.038) for {'randomforestclassifier__n_estimators': 25}
0.259 (+/-0.036) for {'randomforestclassifier__n_estimators': 50}
0.256 (+/-0.039) for {'randomforestclassifier__n_estimators': 100}
0.257 (+/-0.035) for {'randomforestclassifier__n_estimators': 200}
0.259 (+/-0.034) for {'randomforestclassifier__n_estimators': 500}
0.258 (+/-0.035) for {'randomforestclassifier__n_estimators': 1000}

Detailed classification report:

The model is trained on the full development set.
The scores are computed on the full evaluation set.

              precision    recall  f1-score   support

           0       0.90      0.99      0.95     20507
           1       0.78      0.25      0.37      2847

    accuracy                           0.90     23354
   macro avg       0.84      0.62      0.66     23354
weighted avg       0